<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [ ]:
import ai
import numpy as np

In [ ]:
def load_data(file):
    dict = np.load(file, allow_pickle=True)
    return dict

In [ ]:
train_file = 'MNIST/train.npy'
test_file = 'MNIST/test.npy'

In [ ]:
ai.manual_seed(2357)

In [ ]:
class MNIST(ai.Module):
    def __init__(self):
        self.conv1 = ai.Conv2d(1, 8, kernel_size=3, stride=1)
        self.conv2 = ai.Conv2d(8, 16, kernel_size=3, stride=1)
        self.pool1 = ai.Maxpool2d(kernel_size=2, stride=2)
        self.drop1 = ai.Dropout(p=0.75)
        self.fc1 = ai.Linear(2304, 128)
        self.drop2 = ai.Dropout(p=0.5)
        self.fc2 = ai.Linear(128, 10)
        
    def forward(self, x):
        o1 = ai.G.relu(self.conv1(x))
        o2 = ai.G.relu(self.conv2(o1))
        o3 = self.drop1(self.pool1(o2))
        o4 = self.drop2(ai.G.relu(self.fc1(o3)))
        o5 = ai.G.softmax(self.fc2(o4))
        
        return o5

In [ ]:
mnist = MNIST()
print(mnist)

In [ ]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(mnist.parameters(), optim_fn='Adadelta', lr=1e-3)

In [ ]:
train_dict = load_data(train_file)
inputs = train_dict.item()['data']
outputs = train_dict.item()['labels']
del train_dict

In [ ]:
it, epoch = 0, 0
m = 128

In [ ]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = load_data(file)
    inputs = dict.item()['data']
    outputs = dict.item()['labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m].reshape(test_m, 1, 28, 28) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = mnist.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [ ]:
while epoch < 15:
    epoch += 1
    it = 0
    for batch in range(int(len(outputs) / m)):
    # for batch in range(1):
        input = inputs[batch * m : (batch + 1) * m].reshape(m, 1, 28, 28) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = outputs[batch * m : (batch + 1) * m]
        onehot = np.zeros((10, m))
        for _ in range(m):
            onehot[output[_], _] = 1.0
        scores = mnist.forward(input)
        loss = L.loss(scores, onehot)
        loss.backward()
        optim.step()        # update parameters with optimization functions
        optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero
        if it%50 == 0:
            print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0, 0]))
        it += 1
    print('Epoch {} completed. Accuracy: {:.2%} \n'.format(epoch, evaluate()))

In [ ]:
mnist.save()